<a href="https://colab.research.google.com/github/Rushabhbhagat08/Cradit-Card-Default-Prediction/blob/main/Cradit_Card_Default_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - 



##### **Project Type**    - Classification
##### **Contribution**    - Individual
##### **Team Member -** Rushabh Anilrao Bhagat

# **Project Summary -**

Write the summary here within 500-600 words.

# **GitHub Link -**

Provide your GitHub Link here.

# **Problem Statement**


The dataset is details of credit card holders of an "important bank in Taiwan" for the period April to September, 2005. The features available include some basic customer demographics (gender, education, marital status and age), available credit line, their history of payment/default for the six months mentioned (Apr--Sep '05), their bill amounts and their payment amounts for that period and a binary target variable indicating default the following month.

The data was originally studied by the authors of the paper: Yeh, I. C., & Lien, C. H. (2009). The comparisons of data mining techniques for the predictive accuracy of probability of default of credit card clients. Expert Systems with Applications, 36(2), 2473-2480.

I perform some EDA to understand the data and clean the data, engineer relevant features, build predictive models to predict default and perform some statistical analyses to obtain a greater understanding of the features and their interactions. I finish with some business case scenarios where the predictive model could be applied.

# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from numpy import math

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from scipy import stats
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

from sklearn import metrics  
# from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score

# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
# from sklearn.metrics import plot_roc_curve
from sklearn.metrics import precision_recall_curve
# from sklearn.metrics import plot_precision_recall_curve

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier
# from sklearn import metrics

from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score

from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.model_selection import train_test_split as tts

from pprint import pprint
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
!pip install --upgrade xlrd

### Dataset Loading

In [ ]:
# # Load Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load Dataset
file_path=('/content/drive/MyDrive/Classification_project/Credit card default prediction /default of credit card clients (1).xls')
cradit_crd_df=pd.read_excel(file_path)

### Dataset First View

In [ ]:
# Dataset First Look
cradit_crd_df.head()

## Arranged datasets 

In [ ]:
#remove a row 
cradit_crd_df=cradit_crd_df.drop(cradit_crd_df.index[[0,0]],axis=0)

In [ ]:
#Change columns names
update_col=['ID','LIMIT_BAL','SEX','EDUCATION','MARRIAGE','AGE','PAY_1','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6','BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6','PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6','DEF_PAY_NXT_MONTH']

In [ ]:
#update columns name
cradit_crd_df.columns=update_col

In [ ]:
#updated dataset
cradit_crd_df=cradit_crd_df.astype('int')

### Dataset Information

In [ ]:
# the information of all the dataset
cradit_crd_df.info()

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
rows=cradit_crd_df.shape[0]
columns=cradit_crd_df.shape[1]
print(f"The number of rows is {rows} and number of columns is {columns}.")

# Data Cleaning

#### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
condition = bool(cradit_crd_df.duplicated(subset = 'ID').any())

if condition:
    print('There are duplicate IDs')
else:
    print('No duplicate IDs')

#### Identify outliers 

In [ ]:
cradit_crd_df["LIMIT_BAL"].plot(kind="box")
plt.xlabel('Credit limit in NT$', fontweight='bold')
plt.ylabel('# of Customers', fontweight='bold')
plt.show()

In [ ]:
outliers = cradit_crd_df.loc[cradit_crd_df['LIMIT_BAL']>900000]
outliers

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count
cradit_crd_df.isnull().sum()

There is no NaN/NULL record in the dataset, So we dont have to impute any record.

## ***2. Understanding Your Variables***

#### Check numerical columns

In [ ]:
# Dataset Columns
cradit_crd_df.columns

In [ ]:
# Dataset Describe
cradit_crd_df.describe().transpose()

Even though the data description said that the minimum value of pay_i's was -2, all of them have a minimum value of -2. Also marriage has a minimum value of 0 but the description said the minimum value is 1. And education has a maximum value of 6 while it was supposed to have a maximum value of 4. I'll take a closer look later.

I notice that some of the bill_amti's are negative. Although surprising, it is not uncommon to have a negative balance because of merchant refunds or overpayments. The values are a bit on the higher side and I'll take another look later.

## 3. ***Data Wrangling***

### Data Wrangling Code

In [ ]:
# Checking the entries of the ID column
print(cradit_crd_df["ID"].value_counts())
cradit_crd_df.drop(["ID"], axis=1, inplace=True)

In [ ]:
# Write your code to make your dataset analysis ready.
cradit_crd_df.head()

### Variables Description 

This research employed a binary variable, default payment (Yes = 1, No = 0), as the response variable. This study reviewed the literature and used the following 23 variables as explanatory variables:
* X0 (ID): unique id value each row.

* X1 (LIMIT_BAL): Amount of the given credit (NT dollar): it includes both the individual consumer credit and his/her family (supplementary) credit.

* X2 (SEX): Gender (1 = male; 2 = female).

* X3 (EDUCATION): Education (1 = graduate school; 2 = university; 3 = high school; 4 = others).

* X4 (MARRIAGE): Marital status (1 = married; 2 = single; 3 = others).

* X5 (AGE): Age (year).

* X6 (PAY_1) - X11 (PAY_6): History of past payment. We tracked the past monthly payment records (from April to September, 2005) as follows: X6(PAY_1) = the repayment status in September, 2005; X7(PAY_2) = the repayment status in August, 2005; . . ....;X11(PAY_6) = the repayment status in April, 2005. The measurement scale for the repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above.

* X12 (BILL_AMT1)-X17 (BILL_AMT6): Amount of bill statement (NT dollar). X12(BILL_AMT1) = amount of bill statement in September, 2005; X13(BILL_AMT2) = amount of bill statement in August, 2005; . . .; X17(BILL_AMT6) = amount of bill statement in April, 2005.

* X18(PAY_AMT1)-X23 (PAY_AMT6): Amount of previous payment (NT dollar). X18(PAY_AMT1) = amount paid in September, 2005; X19 (PAY_AMT2) = amount paid in August, 2005; . . .;X23(PAY_AMT6) = amount paid in April, 2005.
* DEF_PAY_NXT_MONTH: default payment status next month.


#### Check Unique Values for each variable.

In [ ]:
# Check Unique Values for each variable.
pay=cradit_crd_df[['PAY_1','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']]
pay_melt=pd.melt(pay)
print(pay_melt['value'].value_counts())

## 3. ***Data Wrangling***

### Data Wrangling Code

I will study the relationship between the features and credit card default. Although this is not definitive, it will give me a good understanding of the data. This will also help me make decisions about some of the features having values different than what was provided in the data description.

In [ ]:
# Write your code to make your dataset analysis ready.
def compare_default_across_features(df, col_name, dict_label={}):
    """
    This function quantifies and displays the distribution of default across the various classes of a feature.
    
    INPUT:
    - df - (pd.DataFrame) the dataframe
    - col_name - (str) the column name of the feature being considered
    - dict_label - (dictionary) a dictionary relating the values of the column to what they represent
    
    OUTPUT:
    - Displays a bar plot showing the population distribution by feature values
    - Displays and returns a cross-tab showing the rate of default across each feature value
    - Displays a bar plot showing the population distribution and default by the feature values 
    """

    # Create a cross-tab and rename indices for readability
    cross_tab = pd.crosstab(df["DEF_PAY_NXT_MONTH"], df[col_name], margins=True, normalize=False)

    # new_index=['Not Default','Default']

    new_index = {0: "Non-default proportion", 1: "Default proportion"}
    # new_index={'non_default':0,'default':1}
    new_columns = dict_label

    cross_tab.rename(index=new_index, columns=new_columns, inplace=True)

    # Plot a bar graph showing population distribution by the feature values
    plt.figure(figsize=(10, 5))
    i = cross_tab.shape[1] - 1
    cross_tab.loc["All"][0:i].plot.bar(rot=30, fontsize=14)
    plt.title("Population Distribution by " + col_name, fontsize=20)
    plt.ylabel("count")
    # plt.add_legend()
    plt.show()

    # Normalise the cross-tab and print it
    cross_tab_norm = cross_tab / cross_tab.loc["All"]
    display(cross_tab_norm)

    # Plot a bar graph showing population distribution by the feature values separating the defaulters and non-defaulters
    plt.figure(figsize=(8, 5))
    ax = sns.countplot(x=col_name, hue='DEF_PAY_NXT_MONTH',data=df)
    plt.title(
        "Population Distribution by " + col_name + " with default and non-default",
        fontsize=20,
    )

    # Return the normalised cross-tab
    return cross_tab_norm

In [ ]:
# Define a function to plot bar chart since there will be multiple bars charts to plot
def relationship_bar(column):
   # Get the percentage of default by each group
    default_by_group = pd.crosstab(index=cradit_crd_df['DEF_PAY_NXT_MONTH'],columns = cradit_crd_df[column], normalize = 'columns')
      # Round up to 2 decimal
    default_by_group = default_by_group.apply(lambda x: round(x,2))
    
    labels = default_by_group.columns
    list1 = default_by_group.iloc[0].to_list()
    list2 = default_by_group.iloc[1].to_list()
    list1_name = "No default"
    list2_name = "Has default"
    title = f"Default by {column}"
    xlabel = column
    ylabel = "Default percentage"
    
    fig, ax = plt.subplots(figsize=(10, 5))
    bar_width = 0.5
    
    ax1 = ax.bar(labels,list1, bar_width, label = list1_name)
    ax2 = ax.bar(labels,list2, bar_width, bottom = list1, label = list2_name)

    ax.set_title(title, fontweight = "bold")
    ax.set_xlabel(xlabel, fontweight = "bold")
    ax.set_ylabel(ylabel, fontweight = "bold")
    ax.legend(loc="best")
    
    plt.xticks(list(range(len(labels))), labels,rotation=90)
    plt.yticks(fontsize=9)

    for r1, r2 in zip(ax1, ax2):
        h1 = r1.get_height()
        h2 = r2.get_height()
        plt.text(r1.get_x() + r1.get_width() / 2., h1 / 2., f"{h1:.0%}", ha="center", va="center", color="white", fontsize=9, fontweight="bold")
        plt.text(r2.get_x() + r2.get_width() / 2., h1 + h2 / 2., f"{h2:.0%}", ha="center", va="center", color="white", fontsize=9, fontweight="bold")

    plt.show()

## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

#### Chart - 1

Is default proportion affected by gender?

In [ ]:
# Chart - 1 visualization code
from locale import normalize
# Get the proportion of customers who had default payment in the next month (Oct.2005)? 
# About 22% customers had default payment next month

x=cradit_crd_df['DEF_PAY_NXT_MONTH'].value_counts(normalize=True)
plt.figure(figsize=(8,8))
plt.pie(x, colors=['springgreen', 'coral'], shadow=True, autopct='%1.2f%%', startangle=200)
plt.legend(labels=['Not Default','Default'])
plt.title(" proportion of customers who had default payment in the next month")

#### Chart - 2

In [ ]:
# Get SEX column's distribution. 1: male; 2: female
# No undocumented SEX code
data = cradit_crd_df['SEX'].value_counts().rename_axis('sex').reset_index(name='counts')
data

In [ ]:
# Chart - 2 visualization code
# plot the graph Of SEX column 
plt.figure(figsize=(10, 5))
x= compare_default_across_features(cradit_crd_df, 'SEX', {1: "Male", 2: "Female"})

Although there are more female credit card holders, the default proportion among men is higher. I will do a hypothesis test to see if the difference is statistically significant.
(1: default,0:non_default) 
Is default proportion affected by education?

In [ ]:
#From dataset description: EDUCATION: Education status (1=graduate school; 2 = university; 3 = high school; 4 = others).
print(cradit_crd_df['EDUCATION'].unique())
cradit_crd_df['EDUCATION'].value_counts()

In [ ]:
#But we get 6 values so, we have replace 5,6,0 values 
cradit_crd_df['EDUCATION']=cradit_crd_df['EDUCATION'].replace({0:4,5:4,6:4})
cradit_crd_df['EDUCATION'].value_counts().rename_axis('EDUCATION').reset_index(name='counts')

#### Chart - 3

In [ ]:
# Chart - 3 visualization code
# plot the graph of EDUCATION column 
x = compare_default_across_features(
    cradit_crd_df,
    "EDUCATION",
    {
        1: "Grad School",
        2: "University",
        3: "High School",
        4: "Others",
    },=
)

A quick glance shows that the default proportion decreases with higher education level. This would agree with my intuition because more educated people tend to have higher paying jobs which might make it easier for them to pay back their debts.

As I mentioned earlier, I notice that there are education categories with values 0, 5 and 6 which are not explained by the data description. I could marge those data points, keep them with another cluster. Since there is an Others category, I will cluster them with that category.

#### Chart - 3

Is default proportion affected by marital status?

In [ ]:
# From dataset description: MARRIAGE: Marital status (1=married, 2=single, 3=others), but there is also 0
print(cradit_crd_df['MARRIAGE'].value_counts())

In [ ]:
# we have replace 0 value.
cradit_crd_df["MARRIAGE"] = cradit_crd_df["MARRIAGE"].replace({0:3})
print(cradit_crd_df['MARRIAGE'].value_counts().rename_axis('MARRIAGE').reset_index(name='counts'))

In [ ]:
# Chart - 3 visualization code
# plot the graph of MARRIAGE column
x= compare_default_across_features(
    cradit_crd_df,
    "MARRIAGE",
    {
        1: "Married",
        2: "Single",
        3: "Others",
    },
)

Married people have higher default proportions than single folks. While there are intuitive arguments for and against it, closer inspection is needed. For example, is there a difference between married men and married women?

Also, I notice that there is a marriage category with value 0. Like earlier, since there is an Others category, I will cluster these points with that category. (This is done a bit later.)



#### Chart - 4

Is the proportion of defaults correlated with age?

Since age is a discrete variable, I have to figure out a way to bin the values. There are several ways to do this (including using the frequency and quantiles) but I will choose a simpler and more intuitive way by rounding their age to the lowest multiple of 10.

In [ ]:
# Creating the age bins
cradit_crd_df['age_group'] = cradit_crd_df['AGE']//10
age_group_names = [str(i) + '0s' for i in range(2,8)]
age_dict = dict(zip(range(2,8), age_group_names))
#cc_df['age_bin'] = pd.cut(X['age'], 6, labels=age_group_names)

In [ ]:
# Chart - 4 visualization code
# plot the graph of AGE_GROUP column
age_cross_tab = compare_default_across_features(cradit_crd_df, 'age_group', age_dict)

It is easy to see that default proportion is lowest for people in their 30s and then steadily rises with age.

Is the default proportion affected by credit limit?

Instead of binning the credit limits, I try to see if I can analyse any trends from a density plot.

#### Chart - 5

In [ ]:
# Chart - 5 visualization code
# Make a boxplot to visualize credit limit and default payment next month
# 1: default next month; 0: no default next month
def0 = cradit_crd_df.loc[cradit_crd_df['DEF_PAY_NXT_MONTH'] == 0,'LIMIT_BAL']
def1 = cradit_crd_df.loc[cradit_crd_df['DEF_PAY_NXT_MONTH'] == 1,'LIMIT_BAL']

fig, ax = plt.subplots()
ax.boxplot([def0, def1],  showfliers=False)
ax.set_xticklabels(['No_default',"Default"],fontweight ='bold')
ax.set_ylabel('Credit limit',fontweight ='bold')
ax.set_title('Credit limit & default next month',fontweight ='bold')
plt.show()

Looking at the plot, it seems that people with higher credit limit have significantly lower default proportion. Again, intuitively that is not surprising because the people who have higher credit limits must have displayed long periods of fiscal responsibility to reach that place.

In [ ]:
#see our dataset
cradit_crd_df.head().transpose()

In [ ]:
# Get statistic summary of bill statement columns
# The min numbers are negative
bill = cradit_crd_df[['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']]
bill.describe(include='all').transpose()

#### Chart - 6

In [ ]:
# Chart - 6 visualization code
# Subset a dataframe with the records that have default
has_default = cradit_crd_df[cradit_crd_df['HAS_DEF']== 1]

default_trend = has_default[['PAY_6','PAY_5','PAY_4','PAY_3','PAY_2','PAY_1']].sum(axis=0)

# Draw a line chart to show the trend. The lower the number, the shorter delayed payment
fig,ax = plt.subplots()
ax.plot(default_trend)
plt.xticks(['PAY_6','PAY_5','PAY_4','PAY_3','PAY_2','PAY_1'],['Apr','May','Jun','Jul','Aug','Sep'])

plt.xlabel('Months in 2005',fontweight='bold')
plt.ylabel('Total delayed months',fontweight='bold')
plt.title('Delayed payment trend',fontweight='bold')

plt.show()
     

In [ ]:

# Is there any bill amount that is greater than credit limit?
condition1 = cradit_crd_df['BILL_AMT1'] > cradit_crd_df['LIMIT_BAL'] 
condition2 = cradit_crd_df['BILL_AMT2'] > cradit_crd_df['LIMIT_BAL'] 
condition3 = cradit_crd_df['BILL_AMT3'] > cradit_crd_df['LIMIT_BAL'] 
condition4 = cradit_crd_df['BILL_AMT4'] > cradit_crd_df['LIMIT_BAL'] 
condition5 = cradit_crd_df['BILL_AMT5'] > cradit_crd_df['LIMIT_BAL'] 
condition6 = cradit_crd_df['BILL_AMT6'] > cradit_crd_df['LIMIT_BAL'] 

large_bill = cradit_crd_df[condition1 | condition2 |condition3 | condition4 | condition5 | condition6]
large_bill['DEF_PAY_NXT_MONTH'].value_counts(normalize=True)
# x=cradit_crd_df['DEF_PAY_NXT_MONTH'].value_counts(normalize=True)


In [ ]:
# Create a new column "HAS_DEF" to indicate customers who have at least 1 default payment from PAY_1 to Pay_6
# 0 : no default ; 1: has default

def_condition =(cradit_crd_df.PAY_1>1) | (cradit_crd_df.PAY_2>1) | (cradit_crd_df.PAY_3>1) | (cradit_crd_df.PAY_4>1) | (cradit_crd_df.PAY_5>1) | (cradit_crd_df.PAY_6>1)
cradit_crd_df.loc[def_condition, "HAS_DEF"] = 1
cradit_crd_df.loc[cradit_crd_df.HAS_DEF.isna(), "HAS_DEF"] = 0


In [ ]:
#Create a new column see our dataset
cradit_crd_df.head()

#### Chart 7- Scatter Plot

In [ ]:
# Chart - 7 visualization code
# Plotting the bill amount density plots and their scatter plots
sns.pairplot(cradit_crd_df, vars=cradit_crd_df.columns[11:17], kind='scatter',hue= 'HAS_DEF')

The distribution of the bill amounts are skewed. If the model assumptions require symmetric/normal distributions, a log transformation or a Box-Cox transformation might be warranted.

I will be using a MinMaxScaler later to scale the data given the presence of a lot of outliers.

#### Chart - 8 - Correlation Heatmap

In [ ]:
# chart-8 -- correlation Heatmap
corr = cradit_crd_df.corr()
plt.figure(figsize=(18, 15))
sns.heatmap(corr, annot=True, vmin=-1.0, cmap='mako')
plt.title("Correlation Heatmap")
plt.show()

## ***5. Hypothesis Testing***

## Statistical Inference

###Is Default Affected by Gender?

Does gender affect the default rate? I will try to answer this question with a hypothesis test. As is customary, I'll use a significance level of α=0.05
. Then the bounds of the confidence interval are given by [α/2,1-α/2]=[0.25,0.975].
 
 


In [ ]:
alpha = 0.05
lb, ub = (alpha / 2), 1 - (alpha / 2)
ci_bounds = [lb, ub]

I want to test whether the proportion <b>Pm</b>
 of men defaulting is the same as the proportion <b>Pw</b>
 of women defaulting. Hence, my test statistic is the difference between <b>Pw</b>
 and <b>Pm</b>
.

I state the null Hypothesis and alternate Hypothesis.
* H0: Pm=Pw

* Ha: Pm≠Pw


### Hypothetical Statement 

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

In [ ]:
# Perform Statistical Test to obtain P-Value
ttest, pval = stats.ttest_ind(cradit_crd_df[cradit_crd_df['SEX'] == 1]['HAS_DEF'], cradit_crd_df[cradit_crd_df['SEX'] == 2]['HAS_DEF'], equal_var=False)

In [ ]:
# to check the Pval is grater then 0.05
print(pval)
if pval<0.05:
  print("Null Hypothesis is rejected")
else:
  print("Alternate Hypothesis is rejected")  

Based on the results the null hypotheis of Pm=Pw is rejected. 

## ***6. Feature Engineering & Data Pre-processing***

### 1. Handling Missing Values

Based on my understanding of the data and conclusions from EDA, I engineer a few features.

Having engineered the features, I see if some of the aggregated columns are correlated with default. A quick look does show some relation although I would have to do some statistical tests to see if that is significant.

I draw a scatter plot of average payment amount and average bill amount and notice that there is a marked difference in slope in the line of best fit for the default and non-default class.

I notice that about one third of the people whose average bill amount was more than the credit limit defaulted. As a result, I engineer a feature called overdraft which takes value 1 if the user defaulted at any point in the past 6 months.

#### What all missing value imputation techniques have you used and why did you use those techniques?

### 3. Categorical Encoding

This is the original data set, along with some minor data adjustments discussed earlier and the one-hot encoding of the marriage feature.

This dataset contains features that avoid multicollinearity in the model (and hence, statistically significant non-zero coefficients in a logistic model). I decide on this list by looking at the vif and deviance reduction and also at the feature importances of the models. The dataset is then scaled and the training set balanced.



### 4. Feature Manipulation & Selection

#### 1. Feature Manipulation

In [ ]:
# Manipulate Features to minimize feature correlation and create new features

# Define predictor variables and target variable
X = cradit_crd_df.drop(columns=['DEF_PAY_NXT_MONTH'])
y = cradit_crd_df['DEF_PAY_NXT_MONTH']

# Save all feature names as list
feature_cols = X.columns.tolist() 
#print(feature_cols)
# Extract numerical columns and save as a list for rescaling
X_num = X.drop(columns=['SEX', 'EDUCATION', 'MARRIAGE', 'AGE'])
num_cols = X_num.columns.tolist() 
print(num_cols)

### 2. Data Splitting

In [ ]:
#define a function that count for imbalances

def data_split(x,y,imbalance=False):
  '''
This function will split the data according to the imbalance in the data set 
if imbalance is there in then use SMOTE Analysis   '''

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,shuffle=True, stratify=y, random_state=42)
  if imbalance:
    from imblearn.over_sampling import SMOTE
    sm = SMOTE(random_state = 42)
    X_train, y_train = sm.fit_resample(X_train, y_train.ravel())  
  return X_train, X_test, y_train, y_test

### 6. Data Scaling

In [ ]:
# Define function to rescale training data using StandardScaler
def rescaling(X_train, X_test, numerical_cols):
    
    # Make copies of dataset
    X_train_std = X_train.copy()
    X_test_std = X_test.copy()
    
    # Apply standardization on numerical features only
    for i in numerical_cols:
        scl = StandardScaler().fit(X_train_std[[i]])     # fit on training data columns
        X_train_std[i] = scl.transform(X_train_std[[i]]) # transform the training data columns
        X_test_std[i] = scl.transform(X_test_std[[i]])   # transform the testing data columns
    
    return X_train_std,X_test_std

In [ ]:
def precision_recall(model,X_test,y_test):
  y_pred=model.predict(X_test)
  tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  F1 = 2 * (precision * recall) / (precision + recall)
  print(f'Precision:{precision:.3f}\nRecall:{recall:.3f}\nF1 score:{F1:.3f}')

## ***7. ML Model Implementation***

### ML Model - 1

In [ ]:
# ML Model - 1 Implementation
def logistic_regression(imbalance=False):
  X_train, X_test, y_train, y_test=data_split(X,y,imbalance=imbalance)
  X_train_std,X_test_std=rescaling(X_train,X_test,numerical_cols = num_cols)
# Fit the Algorithm
  clf_lr = LogisticRegression(random_state=42)
  clf_lr.fit(X_train_std, y_train)
  scores = cross_val_score(clf_lr, X_train_std, y_train, scoring ="roc_auc", cv = 5)
  roc_auc_lr = np.mean(scores)*100
  # print(roc_auc_lr)

  if imbalance:
    return "Logistic Regression","With SMOTE",roc_auc_lr
  else:
    return "Logistic Regression","Without SMOTE",roc_auc_lr

# Predict on the model    
model_result=[]
model_result.append(logistic_regression())    
model_result.append(logistic_regression(imbalance=True))
pd.DataFrame(model_result,columns=['Model','SMOTE','ROC_AUC'])

In [ ]:
# Randomized search for the best C parameter
# Split data with SMOTE 
X_train, X_test, y_train, y_test = data_split(X, y, imbalance = True) 

# Rescale data
X_train_std, X_test_std = rescaling(X_train, X_test, numerical_cols = num_cols)
# X_train_std, X_test_std=X_train_std.astype('int')

logistic = LogisticRegression(solver='saga', tol=1e-2, max_iter=200,random_state=42)
distributions = dict(C=uniform(loc=0, scale=4), penalty=['l2', 'l1','elasticnet'])
clf = RandomizedSearchCV(logistic, distributions, random_state=42)

lr_best= clf.fit(X_train_std, y_train)   
#print(distributions)
print(lr_best.best_params_)

In [ ]:
# check the model AUC_ROC Score
scores = cross_val_score(lr_best, X_train_std, y_train, scoring ="roc_auc", cv = 5)
roc_auc_lr = np.mean(scores)
print(f'Roc_Auc score for the Logistic regression with SMOTE :{roc_auc_lr}')

In [ ]:
# check prrecision recall score
precision_recall(lr_best,X_test_std,y_test)     


1. **Accuracy**: The propertion of the total number of prediction are correct.
2. **Precision**: The proportion of positive prediction that are actually correct.
3. **Recall** : The proportion of positive values correctly predicted as such.

**In this Project**
1. **Accuracy**: Overall how often the model predict corrrectly defaulters or not defaulters.
2. **Precision**: when the model predect defaults: How often to correct?
3. **Recall**: The propertion of actual defaulters that the model will correctly predect such as.  

**which matrics shhuch I use?**

1. **False Positive:** A person who will pay predect as defaulters.
2. **false negative:**  A person who default predict as payer

**False negative arde worse==>look for aa better recall**

### ML Model - 2

In [ ]:
# ML Model - 2 Implementation
def RandomForest(imbalance=False):
   X_train, X_test, y_train, y_test=data_split(X,y,imbalance=imbalance)
   X_train_std,X_test_std=rescaling(X_train,X_test,numerical_cols = num_cols)
   clf_lr = RandomForestClassifier(random_state=42)
   clf_lr.fit(X_train_std, y_train)
   scores = cross_val_score(clf_lr, X_train_std, y_train, scoring ="roc_auc", cv = 5)
   roc_auc_lr = np.mean(scores)*100

   if imbalance:
     return "Random Forest","With SMOTE",roc_auc_lr
   else:
     return "Random Forest","Without SMOTE",roc_auc_lr
    
model_result=[]
model_result.append(RandomForest())    
model_result.append(RandomForest(imbalance=True))
pd.DataFrame(model_result,columns=['Model','Smote','ROC_AUC'])

In [ ]:
# Split data with SMOTE
X_train_sm, X_test, y_train_sm, y_test = data_split(X, y, imbalance = True)

In [ ]:
# Create parameter grid  
param_grid = {
    'max_depth': [60, 90, 110],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300]
}

# Instantiate the model
clf_rf = RandomForestClassifier()

# Instantiate grid search model
grid_search = GridSearchCV(estimator = clf_rf, param_grid = param_grid,    
                          cv = 2, n_jobs = -1, verbose = 1)

# Fit grid search to the data
grid_search.fit(X_train_sm, y_train_sm)
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
# check the model AUC_ROC Score
rf_best = RandomForestClassifier(**grid_search.best_params_)   
rf_best.fit(X_train_sm,y_train_sm)

scores_best = cross_val_score(rf_best, X_train_sm, y_train_sm, scoring ="roc_auc", cv = 3)
roc_auc_best = np.mean(scores_best)*100

print(f'ROC_AUC training score after tuning for Random Forest: {roc_auc_best:.3f}')

In [ ]:
print("The F1 score,Precision and Recall value for Random Forest :")
precision_recall(rf_best,X_test,y_test)
# precision_score(X_test,y_test)

### ML Model - 3

In [ ]:
# ML Model - 3 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)
def xgboost(imbalance=False):
   X_train, X_test, y_train, y_test=data_split(X,y,imbalance=imbalance)
   X_train_std,X_test_std=rescaling(X_train,X_test,numerical_cols = num_cols)
   clf_lr = XGBClassifier(random_state=42)
   clf_lr.fit(X_train_std, y_train)
  #  model_xgb.fit(X_train, y_train)
   scores = cross_val_score(clf_lr, X_train_std, y_train, scoring ="roc_auc", cv = 5)
   roc_auc_lr = np.mean(scores)*100
  #  print(roc_auc_lr)
   if imbalance:
     return "XGBOOST","With SMOTE",roc_auc_lr
   else:
     return "XGBOOST","Without SMOTE",roc_auc_lr    
model_result=[]
# y=y.astype('int')
model_result.append(xgboost())    
model_result.append(xgboost(imbalance=True))
pd.DataFrame(model_result,columns=['Model','SMOTE','ROC_AUC'])

In [ ]:
# Split data with SMOTE
X_train_sm, X_test, y_train_sm, y_test = data_split(X, y, imbalance = True)

In [ ]:
# from skopt.space import Bayes searchCV
# from skopt.space import real,categorical,integer
params = { 
    'gamma':0,
    'learning_rate':0.01, 
    'max_depth':3, 
    'colsample_bytree':0.6,
    'subsample':0.8,
    'scale_pos_weight':3.5,
    'n_estimators':1000,
    'objective':'binary:logistic', 
    'reg_alpha':0.3    
}

clf_xgb=XGBClassifier(**params)
scores_best = cross_val_score(clf_xgb, X_train_sm, y_train_sm, scoring ="roc_auc", cv = 3)
roc_auc_best = np.mean(scores_best)*100
print(f'ROC_AUC training score after tuning for initial parameter in XGBOOST: {roc_auc_best:.3f}')

In [ ]:
n_estimators = np.arange(200,1000,200)

# Minimum loss reduction required to make a further partition on a leaf node of the tree
# The larger gamma is, the more conservative the algorithm will be
gamma = np.arange(0.1,0.6,0.1)

# Default 0.3, range(0,1)
learning_rate = np.arange(0.1,0.6,0.1)

# Maximum number of levels in tree
max_depth = list(range(3,8,1))

# Subsample ratio of the training instances.Range(0,1)
subsample = np.arange(0.5,0.9,0.1)

# Subsample ratio of columns when constructing each tree. Range(0,1)
colsample_bytree = np.arange(0.5,0.9,0.1)

# Control the balance of positive and negative weights
# Sum(negative instances) / sum(positive instances)
scale_pos_weight = [1,3.5]


# Create the random grid
random_grid_xgb = {'n_estimators': n_estimators,
                   'gamma': gamma,
                   'learning_rate':learning_rate,
                   'max_depth': max_depth,
                   'subsample':subsample,
                   'colsample_bytree':colsample_bytree,
                   'scale_pos_weight':scale_pos_weight
                  }
print(random_grid_xgb)     

In [ ]:

xgboost = XGBClassifier()
xgb_random = RandomizedSearchCV(estimator = xgboost, 
                                param_distributions = random_grid_xgb, 
                                n_iter = 10, 
                                cv = 2, 
                                verbose=1, 
                                random_state=42, 
                                n_jobs = -1,
                                scoring ='roc_auc')


xgb_random.fit(X_train_sm, y_train_sm)   
xgb_random.best_params_, xgb_random.best_score_

print(xgb_random.best_params_,xgb_random.best_score_)

In [ ]:
# check the model F1 score, Precision and Recall Score
print("The F1 score, Precision and Recall for XGBOOST is :")
precision_recall(xgb_random,X_test,y_test)


##Plot Confusion Matrix 

In [ ]:
def plot_feat_imp(X_train, model):
    """
    Prints the model feature importances
    """
    feat_imp = pd.DataFrame(
        {"Feature": X_train.columns, "Feature Importance": model.feature_importances_}
    )
    feat_imp.sort_values(by="Feature Importance", ascending=False, inplace=True)
    plt.figure(figsize=(10, 5))
    plt.title(f"Feature Importance of {model}", fontsize=14)
    ax = sns.barplot(x="Feature", y="Feature Importance", data=feat_imp)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    plt.show()
    
def get_model_results(X_train, X_test, y_train, y_test, model, feat_imp=False):
    """
    Prints the training and test metrics and plots the confusion matrices.
    Also plots the feature importance based on the user choosing the parameter variable,
    and returns the evaluation metrics.
    """
    # Fit training model to training set
    model.fit(X_train, y_train)

    # Obtain the predicted values and probabilities from the model
    y_train_pred = model.predict(X_train)# y-train predict model
    y_train_pred_probs = model.predict_proba(X_train)# y- train probability of model predict

    y_test_pred = model.predict(X_test)# y-test predicted
    y_test_pred_probs = model.predict_proba(X_test)#y-test predict probility 

    # Get confusion matrix
    train_matrix = metrics.confusion_matrix(y_train, y_train_pred)# y-train,y-train-pred confusion matrix
    test_matrix = metrics.confusion_matrix(y_test, y_test_pred) #y-test,y-test-pred confusion matrix

    # Get F1 score
    f1_train = metrics.f1_score(y_train, y_train_pred)
    f1_test = metrics.f1_score(y_test, y_test_pred)

    # Print the train and test roc_auc_score
    #and test confusion matrix
    print(
        f"ROC_AUC_Score of train set is {metrics.roc_auc_score(y_train, y_train_pred_probs[:, 1])}."
    )
    print(
        f"ROC_AUC_Score of test set is {metrics.roc_auc_score(y_test, y_test_pred_probs[:, 1])}."
    )
    #print("Confusion matrix of test set\n", confusion_matrix(y_test, y_test_pred))
    #print(
    #    "Classification Report of test set\n",
    #    metrics.classification_report(y_test, y_test_pred),
    #)
    
    # Display scores
    print(f"F1 of train set is {f1_train.round(4)}.")
    print(f"F1 of test set is {f1_test.round(4)}.")

    # Plot training and test confusion matrices
    plt.figure(figsize=(12, 4))
    names = ["True Neg", "False Pos", "False Neg", "True Pos"]
    train_percent = [
        "{0:.2%}".format(value)
        for value in train_matrix.flatten() / np.sum(train_matrix)
    ]
    labels = [f"{v1}\n{v2}" for v1, v2 in zip(names, train_percent)]
    labels = np.asarray(labels).reshape(2, 2)
    plt.subplot(1, 2, 1)
    sns.heatmap(train_matrix, annot=labels, fmt="", cmap="Blues")
    plt.title("Train Set\n")

    names = ["True Neg", "False Pos", "False Neg", "True Pos"]
    test_percent = [
        "{0:.2%}".format(value) for value in test_matrix.flatten() / np.sum(test_matrix)
    ]
    labels = [f"{v1}\n{v2}" for v1, v2 in zip(names, test_percent)]
    labels = np.asarray(labels).reshape(2, 2)
    plt.subplot(1, 2, 2)
    sns.heatmap(test_matrix, annot=labels, fmt="", cmap="Blues")
    plt.title("Test Set\n")

    plt.subplots_adjust(wspace=0.2)

    #plt.figure(figsize=(7, 4))
    #plot_roc_curve(model, X_test, y_test)
    #plt.show()

    if feat_imp == True:
        plot_feat_imp(X_train, model)
    elif feat_imp == "LR":
        # Get feature coefficients
        importance = model.coef_[0]

        df_feat_imp = pd.DataFrame(
            columns=["Feature", "Feature Importance (Value of Coefficient)"]
        )
        
        # Attach feature coefficients to labels
        for i, v in enumerate(importance):
            df_feat_imp = df_feat_imp.append(
                {
                    "Feature": X_train.columns[i],
                    "Feature Importance (Value of Coefficient)": v,
                },
                ignore_index=True,
            )

            #print(f"{X_train.columns[i]} Feature: %0d, Score: %.5f" % (i, v))
        
        # Sort them by their absolute values in descending order
        df_feat_imp = df_feat_imp.reindex(
            df_feat_imp["Feature Importance (Value of Coefficient)"]
            .abs()
            .sort_values(ascending=False)
            .index
        )
        
        # Plot a bargraph
        plt.figure(figsize=(10, 5)) # plot the grah
        plt.title(f"Feature Importance of {model}", fontsize=14)
        ax = sns.barplot(x="Feature", y="Feature Importance (Value of Coefficient)", data=df_feat_imp)
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
        plt.show()
        
    else:
        pass
    
    return metrics.accuracy_score(y_test, y_test_pred),\
           metrics.precision_score(y_test, y_test_pred),\
           metrics.recall_score(y_test, y_test_pred),\
           metrics.f1_score(y_test, y_test_pred),\
           metrics.roc_auc_score(y_test, y_test_pred_probs[:, 1])



### Logistic Regression Confusion matrix

In [ ]:
# Instantiate a Logistic Regression model
log_model = LogisticRegression(random_state=0, max_iter=200)

# Train the model on the original data set and diplay the performance
acc, prec, rec, f1, auc = get_model_results(X_train, X_test, y_train, y_test, log_model)
# log_model.set_title('logistic regression')

### Random Forest Confusion matrix

In [ ]:
# Random forest condusion matrix
rf_model = RandomForestClassifier(random_state=0)
acc, prec, rec, f1, auc = get_model_results(X_train, X_test, y_train, y_test, rf_model)

### XGBoost Confusion matrix

In [ ]:
# Instantiating the best model
clf_lr = XGBClassifier(random_state=42)

# I am using resampled data as I am not using class_weight ={0:1, 1:4} to balance

acc, prec, rec, f1, auc = get_model_results(X_train, X_test, y_train, y_test, clf_lr)

### Implementt Dummy Model

In [ ]:
# implement dummy model
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="stratified")
dummy_clf.fit(X_train, y_train)
DummyClassifier(strategy='stratified')
y_pred_dummy = dummy_clf.predict(X_test)

print('Dummy model:')
precision_recall(dummy_clf, X_test, y_test)

In [ ]:
# Compute precision, recall and threshold of Random Forest

y_predict_rf = rf_best.predict_proba(X_test)

y_scores_rf = y_predict_rf[:,1]
#print(y_scores_rf)
precisions, recalls, thresholds = precision_recall_curve(y_test, y_scores_rf)
#print(precisions)
#print(recalls)
#print(thresholds)
recalls_80 = recalls[np.argmin(recalls >= 0.80)]# Recommend recall score = 0.8
precision_80 = precisions[np.argmin(recalls >= 0.80)]
threshold_80_recall = thresholds[np.argmin(recalls >= 0.80)]

thresholds = np.append(thresholds, 1)

recalls_80, precision_80, threshold_80_recall

In [ ]:
# Plot feature importance of winner model - Random Forest

fea_df = pd.DataFrame({'Feature': feature_cols, 'Feature importance': rf_best.feature_importances_})
fea_df = fea_df.sort_values(by='Feature importance')

figure, ax = plt.subplots(figsize = (10,8))
fea_df.plot.barh(x='Feature',y='Feature importance', ax=ax)
plt.title('Features importance',fontsize=14)
     

## ***8.*** ***Future Work (Optional)***

### ***Congrats! Your model is successfully created and ready for deployment on a live server for a real user interaction !!!***

# **Conclusion**

Logistic Regression model has the highest recall but the lowest precision, if the business cares recall the most, then this model is the best candidate. If the balance of recall and precision is the most important metric, then Random Forest is the ideal model. Since Random Forest has slightly lower recall but much higher precision than Logistic Regression, I would recommend Random Forest.

### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***